# KNN exercise with a household financial data

## Introduction

- any household maps transactions to categories. we want to see if the predictor can identify the most accurate category.

## Step 1: Read the data into Pandas

In [1]:
# read the data into a DataFrame named nba
import pandas as pd
import numpy as np
from itertools import chain
from numpy import ndenumerate 
import ast

path = 'C:/Users/saang/Documents/Home_Metrics/'
url = path + 'Transactions-2015-2016Q2.csv'
tran = pd.read_csv(url)

In [2]:
# examine the columns names to see what's there
tran.columns

Index([u'Date', u'Category', u'Description', u'Bill', u'Account', u'In/Out',
       u'Actuals/Budget', u'addt'l info', u'CatBill', u'CatBillCodeZZZ',
       u'dayZZZ', u'monthZZZ', u'AccountCodeZZZ', u'InOutCodeZZZ',
       u'ActBudCodeZZZ'],
      dtype='object')

In [3]:
#tran.CatBill.count()
#tran.groupby(['CatBill']).count()
tran.CatBill.value_counts()

Restaurants/Dining:N/A:PAYMENT                  740
Transfers:N/A:Transfers                         222
Clothing/Shoes:N/A:PAYMENT                      214
Groceries:QFC:PAYMENT                           169
Travel:N/A:PAYMENT                              146
Groceries:Costco:PAYMENT                        142
Gasoline/Fuel:Gas:PAYMENT                       132
Groceries:N/A:PAYMENT                           126
Business Miscellaneous:N/A:PAYMENT              119
Healthcare/Medical:N/A:PAYMENT                   92
Paychecks/Salary:SAL-SA:INCOME                   79
Entertainment:N/A:PAYMENT                        77
Parking:N/A:PAYMENT                              74
Groceries:Amazon.com:PAYMENT                     66
Service Charges/Fees:N/A:PAYMENT                 66
Paychecks/Salary:SAL-SV:INCOME                   60
Healthcare/Medical:Walgreens:PAYMENT             54
Home Improvement:NewHouse:PAYMENT                48
ATM/Cash Withdrawals:N/A:PAYMENT                 47
Child/Depend

In [4]:
#tran.head()
tran.shape

(4632, 15)

## Getting Description column in an string



In [5]:
newdf = tran['Description'].map(str)
#print (newdf)
out=""
for sent in newdf.index:
    #print newdf[sent]  
    out+= " "
    out+= newdf[sent]
#print (out)

out = out.replace('xxx','').replace('%%','')
out = out.replace('xxx','').replace('xx','')


## Remove Accents

In [6]:
import unicodedata

def remove_accents(input_str):
    decoded_str = input_str.decode("windows-1252")
    nfkd_form = unicodedata.normalize('NFKD', unicode(decoded_str))
    #nfkd_form = nfkd_form.encode("utf8","ignore")
    #decoded_str = nfkd_form.decode("windows-1252")
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    
    return only_ascii

input_string = remove_accents(out)

#print(input_string)
type(input_string)

str

## Tokenize

In [7]:
import nltk
#nltk.download()

from nltk.tokenize import sent_tokenize

sent_tokens = sent_tokenize(input_string)

#print (sent_tokens)

for sent in sent_tokens:
    print("--- sentence: {}".format(sent))

--- sentence:  PAYMENT - THANK YOU Deposit #x84 - State Department FCU Mobile Alexandria VA  Check hold release 04/30/15 0.00  Check hold release 05/01/15 40.00  Check hold release 05/06/15 450.00 Deposit Home Banking Transfer - From Share 0000 SDFCU Online 02/23/2015 19:08 9664 Deposit VANGUARD GROUP - TYPE: DISBRSMNT ID: 6884 CO: VANGUARD GROUP  ACH ECC PPD  ACH Trace 1464 Deposit Home Banking Transfer - From Share 0001 SDFCU Online 07/06/2015 10:37 3816 Deposit #x51 - State Department FCU Mobile Alexandria VA  Check hold release 04/28/15 0.00  Check hold release 04/29/15 40.00  Check hold release 05/04/15 150.00 Online Banking transfer from SAV 4195                       Confirmation# 6408 PAYMENT RECEIVED - THANK YOU Online Banking transfer from CHK 0403                       Confirmation# 4753 IRS  TREAS 310   DES:  TAX REF  ID:0928           INDN:ANGULO SANTIAGO & SAN  CO ID:9111 ELECTRONIC PAYMENT RECEIVED-THA Online Banking transfer from CHK 0403                       Confirmat

## Word Tokenize

In [8]:
import string
from nltk.tokenize import RegexpTokenizer
#s = "Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\n\nThanks."
#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
tokenizer = RegexpTokenizer('\s+', gaps=True)
#tokens_lower = [map(string.lower, input_string)]
words_tokens = tokenizer.tokenize(input_string)

#tokens_lower = [map(string.lower, sent_tokens)]
tokens_lower = [map(string.lower, words_tokens)]

#print words_tokens

#print tokens_lower

## Make string lowercase and removing stopwords

In [9]:
import string
# list found at http://www.textfixer.com/resources/common-english-words.txt
# 'not' has been removed (do you know why ?)

stopwords_ = "a,able,about,across,after,all,almost,also,am,among,an,and,any,\
are,as,at,be,because,been,but,by,can,could,dear,did,do,does,either,\
else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,\
how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,\
me,might,most,must,my,neither,no,of,off,often,on,only,or,other,our,\
own,rather,said,say,says,she,should,since,so,some,than,that,the,their,\
them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,\
what,when,where,which,while,who,whom,why,will,with,would,yet,you,your,%%]".split(',')

#print("--- stopwords in english: {}".format(stopwords_))


punctuation_ = set(string.punctuation)
#print("--- punctuation: {}".format(string.punctuation))

def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

#newstring = list(filter_tokens(tokens))
tokens_filtered = map(filter_tokens, tokens_lower)
print type(tokens_filtered)
print (tokens_filtered)

<type 'list'>
[['payment', 'thank', 'deposit', '#x84', 'state', 'department', 'fcu', 'mobile', 'alexandria', 'va', 'check', 'hold', 'release', '04/30/15', '0.00', 'check', 'hold', 'release', '05/01/15', '40.00', 'check', 'hold', 'release', '05/06/15', '450.00', 'deposit', 'home', 'banking', 'transfer', 'share', '0000', 'sdfcu', 'online', '02/23/2015', '19:08', '9664', 'deposit', 'vanguard', 'group', 'type:', 'disbrsmnt', 'id:', '6884', 'co:', 'vanguard', 'group', 'ach', 'ecc', 'ppd', 'ach', 'trace', '1464', 'deposit', 'home', 'banking', 'transfer', 'share', '0001', 'sdfcu', 'online', '07/06/2015', '10:37', '3816', 'deposit', '#x51', 'state', 'department', 'fcu', 'mobile', 'alexandria', 'va', 'check', 'hold', 'release', '04/28/15', '0.00', 'check', 'hold', 'release', '04/29/15', '40.00', 'check', 'hold', 'release', '05/04/15', '150.00', 'online', 'banking', 'transfer', 'sav', '4195', 'confirmation#', '6408', 'payment', 'received', 'thank', 'online', 'banking', 'transfer', 'chk', '0403',

## 1.3. Stemming and lemmatization

**Stemming** means reducing each word to a **stem**. That is, reducing each word in all its diversity to a root common to all its variants.

In [10]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

stemmer_porter = PorterStemmer()
tokens_stemporter = [map(stemmer_porter.stem, sent) for sent in tokens_filtered]
print("--- sentence tokens (porter): {}".format(tokens_stemporter[0]))


--- sentence tokens (porter): [u'payment', u'thank', u'deposit', u'#x84', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/30/15', u'0.00', u'check', u'hold', u'releas', u'05/01/15', u'40.00', u'check', u'hold', u'releas', u'05/06/15', u'450.00', u'deposit', u'home', u'bank', u'transfer', u'share', u'0000', u'sdfcu', u'onlin', u'02/23/2015', u'19:08', u'9664', u'deposit', u'vanguard', u'group', u'type:', u'disbrsmnt', u'id:', u'6884', u'co:', u'vanguard', u'group', u'ach', u'ecc', u'ppd', u'ach', u'trace', u'1464', u'deposit', u'home', u'bank', u'transfer', u'share', u'0001', u'sdfcu', u'onlin', u'07/06/2015', u'10:37', u'3816', u'deposit', u'#x51', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/28/15', u'0.00', u'check', u'hold', u'releas', u'04/29/15', u'40.00', u'check', u'hold', u'releas', u'05/04/15', u'150.00', u'onlin', u'bank', u'transfer', u'sav', u'4195', u'confirmation#', u'6408',

In [11]:
stemmer_snowball = SnowballStemmer('english')
tokens_stemsnowball = [map(stemmer_snowball.stem, sent) for sent in tokens_filtered]
#print("--- sentence tokens (snowball): {}".format(tokens_stemsnowball[0]))

## dedup

In [12]:
#df = pd.DataFrame(tokens_filtered)
df = pd.DataFrame(tokens_stemporter)
df.count()
temp = df.T
temp.count()
#df.head()

0    31416
dtype: int64

In [13]:
deduped_df = temp.drop_duplicates()
deduped_df.count()
#deduped_df
features=deduped_df.T
#features.head()
features



,0,1,2,3,4,5,6,7,8,9,...,31271,31272,31283,31291,31307,31317,31335,31347,31348,31349
0,payment,thank,deposit,#x84,state,depart,fcu,mobil,alexandria,va,...,06/30,9531,x6506,x6128,06/30/16,0737,06/29/16,caff,umbria,x4723


### Make row 0 to columns

In [14]:
df=pd.DataFrame(columns=features.loc[0,])
#df.head()
print df.shape
df.head()


(0, 5031)


,payment,thank,deposit,#x84,state,depart,fcu,mobil,alexandria,va,...,06/30,9531,x6506,x6128,06/30/16,0737,06/29/16,caff,umbria,x4723


## Fill DataFrame with zeros

In [15]:
import numpy as np
#df.shape
a = np.zeros((4632,5031))

df=pd.DataFrame(data=a,columns=features.loc[0,])
df.shape
#df.head()



(4632, 5031)

## Combine data Frames

In [16]:
frames=[tran, df]
#df1 = pd.concat(frames, )
result = pd.concat([tran, df], axis=1)
#result



## Count words in description and update the features

### Create a DataFrame with only the Descriptions, lowercase, tokenize

In [17]:
desc= tran['Description']
descdf = pd.DataFrame(desc)
descdf.Description = descdf.Description.str.lower()

import unicodedata

def remove_accents(input_str):
    decoded_str = input_str.decode("windows-1252")
    nfkd_form = unicodedata.normalize('NFKD', unicode(decoded_str))
#    nfkd_form = unicodedata.normalize('NFKD', unicode(input_str))
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

#descdf.Description = 
#input_string = remove_accents(cont)
#input_string = str(input_string)
#print(input_string)


from nltk.tokenize import RegexpTokenizer
#s = "Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\n\nThanks."
#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
tokenizer = RegexpTokenizer('\s+', gaps=True)
#tokens_lower = [map(string.lower, input_string)]

for index, row in descdf.iterrows():
    descdf.loc[index,'Description'] = descdf.loc[index,'Description'].replace('xxx','').replace('%%','')
    descdf.loc[index,'Description'] = descdf.loc[index,'Description'].replace('xx','')
    descdf.loc[index,'Description'] = remove_accents(descdf.loc[index,'Description'])
    descdf.loc[index,'Description'] = tokenizer.tokenize(descdf.loc[index,'Description'])

#descdf


## remove stopwords

In [18]:
import string
# list found at http://www.textfixer.com/resources/common-english-words.txt
# 'not' has been removed (do you know why ?)

stopwords_ = "a,able,about,across,after,all,almost,also,am,among,an,and,any,\
are,as,at,be,because,been,but,by,can,could,dear,did,do,does,either,\
else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,\
how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,\
me,might,most,must,my,neither,no,of,off,often,on,only,or,other,our,\
own,rather,said,say,says,she,should,since,so,some,than,that,the,their,\
them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,\
what,when,where,which,while,who,whom,why,will,with,would,yet,you,your,/%/%]".split(',')

#print("--- stopwords in english: {}".format(stopwords_))

#punctuation_ = set(string.punctuation)
#print("--- punctuation: {}".format(string.punctuation))

def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

for index, row in descdf.iterrows():
    descdf.loc[index,'Description'] = filter_tokens(descdf.loc[index,'Description'])

    #return([w for w in descdf.loc[index,'Description'] if not w in stopwords_ and not w in punctuation_])
    
#descdf


In [ ]:
#descdf.loc[index,'Description']
descdf

In [25]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

stemmer_porter = PorterStemmer()
#tokens_stemporter = [map(stemmer_porter.stem, sent) for sent in tokens_filtered]
#print("--- sentence tokens (porter): {}".format(tokens_stemporter[0]))

#words = list()
for index, row in descdf.iterrows():
    #descdf.loc[index,'Description'] = remove_accents(descdf.loc[index,'Description'])
    words = "["+str(row.Description)+"]"
    #words = list(words)
   #print index, words
    #print type(words)
    #for item in words:
        #print item
        #newval = PorterStemmer(item).stem
        #print newval
    items = list(ast.literal_eval(words))
    ###items=list(words)
    #print items
    tokens_stemporter = [map(stemmer_porter.stem, sent) for sent in items]
    descdf.loc[index,'newdesc'] = tokens_stemporter[0]
    print tokens_stemporter

[[u'payment', u'thank']]
[[u'deposit', u'#x84', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/30/15', u'0.00', u'check', u'hold', u'releas', u'05/01/15', u'40.00', u'check', u'hold', u'releas', u'05/06/15', u'450.00']]
[[u'deposit', u'home', u'bank', u'transfer', u'share', u'0000', u'sdfcu', u'onlin', u'02/23/2015', u'19:08', u'9664']]
[[u'deposit', u'vanguard', u'group', u'type:', u'disbrsmnt', u'id:', u'6884', u'co:', u'vanguard', u'group', u'ach', u'ecc', u'ppd', u'ach', u'trace', u'1464']]
[[u'deposit', u'home', u'bank', u'transfer', u'share', u'0001', u'sdfcu', u'onlin', u'07/06/2015', u'10:37', u'3816']]
[[u'deposit', u'#x51', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/28/15', u'0.00', u'check', u'hold', u'releas', u'04/29/15', u'40.00', u'check', u'hold', u'releas', u'05/04/15', u'150.00']]
[[u'onlin', u'bank', u'transfer', u'sav', u'4195', u'confirmation#', u'6408']]
[[u'payme

In [21]:
####NO ####
words = [['payment', 'thank']]
tokens_stemporter = [map(stemmer_porter.stem, sent) for sent in words]
    #descdf.loc[index,'newdesc'] = tokens_stemporter
print tokens_stemporter[0]

### [[u'payment', u'thank']]


[u'payment', u'thank']


In [26]:
descdf.loc[1,'newdesc']


####   "[[u'deposit', u'#x84', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/30/15', u'0.00', u'check', u'hold', u'releas', u'05/01/15', u'40.00', u'check', u'hold', u'releas', u'05/06/15', u'450.00']]"

[u'deposit',
 u'#x84',
 u'state',
 u'depart',
 u'fcu',
 u'mobil',
 u'alexandria',
 u'va',
 u'check',
 u'hold',
 u'releas',
 u'04/30/15',
 u'0.00',
 u'check',
 u'hold',
 u'releas',
 u'05/01/15',
 u'40.00',
 u'check',
 u'hold',
 u'releas',
 u'05/06/15',
 u'450.00']

## Count words per row

In [ ]:
descdf.loc[1,'Description']

In [27]:
wordfreq = []
wordlist = ""
def rocks(wordlist):
    for w in wordlist:
        wordfreq.append(wordlist.count(w))
    return (str(zip(wordlist, wordfreq)))
    
countspw = []
desc = []
for desc in descdf.newdesc:
    #desc = desc.replace('[[','').replace(']]','')
    #desc1 = desc.split()
    countspw.append(rocks(desc))
    print desc


# check that the DataFrame and the list of years are the same length
assert (len(descdf) == len(countspw))

#print countspw

# save that list as a new column
result['countspw'] = countspw

#result.head()
result.shape

[u'payment', u'thank']
[u'deposit', u'#x84', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/30/15', u'0.00', u'check', u'hold', u'releas', u'05/01/15', u'40.00', u'check', u'hold', u'releas', u'05/06/15', u'450.00']
[u'deposit', u'home', u'bank', u'transfer', u'share', u'0000', u'sdfcu', u'onlin', u'02/23/2015', u'19:08', u'9664']
[u'deposit', u'vanguard', u'group', u'type:', u'disbrsmnt', u'id:', u'6884', u'co:', u'vanguard', u'group', u'ach', u'ecc', u'ppd', u'ach', u'trace', u'1464']
[u'deposit', u'home', u'bank', u'transfer', u'share', u'0001', u'sdfcu', u'onlin', u'07/06/2015', u'10:37', u'3816']
[u'deposit', u'#x51', u'state', u'depart', u'fcu', u'mobil', u'alexandria', u'va', u'check', u'hold', u'releas', u'04/28/15', u'0.00', u'check', u'hold', u'releas', u'04/29/15', u'40.00', u'check', u'hold', u'releas', u'05/04/15', u'150.00']
[u'onlin', u'bank', u'transfer', u'sav', u'4195', u'confirmation#', u'6408']
[u'payment', u'receiv',

(4632, 5047)

In [28]:
result.loc[1,'countspw']

"[(u'deposit', 1), (u'#x84', 1), (u'state', 1), (u'depart', 1), (u'fcu', 1), (u'mobil', 1), (u'alexandria', 1), (u'va', 1), (u'check', 1), (u'hold', 1), (u'releas', 3), (u'04/30/15', 3), (u'0.00', 3), (u'check', 1), (u'hold', 1), (u'releas', 3), (u'05/01/15', 3), (u'40.00', 3), (u'check', 1), (u'hold', 1), (u'releas', 3), (u'05/06/15', 3), (u'450.00', 3)]"

## fill the features with values

In [29]:
import numpy as np
from itertools import chain
from numpy import ndenumerate 
import ast
#result5 = result.head() 

line=list()
tup=()
for index, row in result.iterrows():
    #line = np.array(row.countspw)
    line = row.countspw
    #print line
    #print type(line)
    items = list(ast.literal_eval(row.countspw))

    for item in items:
        #print "Item = " + str(item)
        tup = tuple(item)
      
        for i,j in enumerate(item):
            #print "index i and j : " + str(index) + " : " + str(i) + " : " + str(j)
            col = str(item[0])
            #print ("colum is " +str(item[0]))
            val = item[1]
            #print ("value is "+ str(item[1]))
                        
            result.loc[index,col] = val
            if col in result.columns:
                a=0
            else:
                print ("index = :" +str(index) + " colum is :" + col + " value is :"+ str(val))
                #print ("colum is " + col)
                #print ("value is "+ str(val))
            if (index == 1000) or (index == 2000) or (index == 3000) or ( index == 4000 ):
                print "1000"


1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [30]:
#result.loc[:,u'payment']
result


,Date,Category,Description,Bill,Account,In/Out,Actuals/Budget,addt'l info,CatBill,CatBillCodeZZZ,...,06/30/16,0737,06/29/16,caff,umbria,x4723,countspw,id:6900,nmbp,zzzgns7jj
0,1/5/2015,Credit Card Payments,PAYMENT - THANK YOU,BOA-SA,Bank of America - Credit Card - BankAmericard ...,CC pmnt,Actuals,0,Credit Card Payments:BOA-SA:CC pmnt,1,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'payment', 1), (u'thank', 1)]",NaN,NaN,NaN
1,4/29/2015,Luis Angulo Padre,Deposit #xxxx84 - State Department FCU Mobile ...,NaN,State Department FCU - Bank - REGULAR SAVINGS,Loan,Actuals,0,Luis Angulo Padre:N/A:Loan,2,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'deposit', 1), (u'#x84', 1), (u'state', 1),...",NaN,NaN,NaN
2,2/23/2015,Transfers,Deposit Home Banking Transfer - From Share 000...,NaN,State Department FCU - Bank - BASIC CHECKING,Transfers,Actuals,0,Transfers:N/A:Transfers,3,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'deposit', 1), (u'home', 1), (u'bank', 1), ...",NaN,NaN,NaN
3,7/6/2015,401K,Deposit VANGUARD GROUP - TYPE: DISBRSMNT ID: x...,NaN,State Department FCU - Bank - BASIC CHECKING,INCOME,Actuals,0,401K:N/A:INCOME,4,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'deposit', 1), (u'vanguard', 1), (u'group',...",NaN,NaN,NaN
4,7/6/2015,Transfers,Deposit Home Banking Transfer - From Share 000...,NaN,State Department FCU - Bank - REGULAR SAVINGS,Transfers,Actuals,0,Transfers:N/A:Transfers,3,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'deposit', 1), (u'home', 1), (u'bank', 1), ...",NaN,NaN,NaN
5,4/27/2015,Transfers,Deposit #xxxx51 - State Department FCU Mobile ...,NaN,State Department FCU - Bank - REGULAR SAVINGS,Transfers,Actuals,0,Transfers:N/A:Transfers,3,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'deposit', 1), (u'#x51', 1), (u'state', 1),...",NaN,NaN,NaN
6,4/27/2015,Transfers,Online Banking transfer from SAV 4195 ...,NaN,Bank of America - Bank - Bank of America Core ...,Transfers,Actuals,0,Transfers:N/A:Transfers,3,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'onlin', 1), (u'bank', 1), (u'transfer', 1)...",NaN,NaN,NaN
7,9/27/2015,Transfers,PAYMENT RECEIVED - THANK YOU,AmexBlue,American Express Cards - Blue for Business,Transfers,Actuals,0,Transfers:AmexBlue:Transfers,5,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'payment', 1), (u'receiv', 1), (u'thank', 1)]",NaN,NaN,NaN
8,4/24/2015,Transfers,Online Banking transfer from CHK 0403 ...,NaN,Bank of America - Bank - Regular Savings,Transfers,Actuals,0,Transfers:N/A:Transfers,3,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'onlin', 1), (u'bank', 1), (u'transfer', 1)...",NaN,NaN,NaN
9,4/22/2015,Taxes,IRS TREAS 310 DES: TAX REF ID:xxxxxxxxxxx...,NaN,Bank of America - Bank - Bank of America Core ...,INCOME,Actuals,0,Taxes:N/A:INCOME,6,...,0.0,0.0,0.0,0.0,0.0,0.0,"[(u'ir', 1), (u'trea', 1), (u'310', 1), (u'des...",NaN,NaN,NaN


In [31]:
result.loc[:,'AccountCodeZZZ':'x4723'].head(10)
#result.loc[:,'payment':'06/30/16']

,AccountCodeZZZ,InOutCodeZZZ,ActBudCodeZZZ,payment,thank,deposit,#x84,state,depart,fcu,...,06/30,9531,x6506,x6128,06/30/16,0737,06/29/16,caff,umbria,x4723
0,10,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21,2,1,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20,3,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20,4,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21,3,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,21,3,1,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2,3,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,5,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## create Every word as a feature

## Step 2: Create X and y

Use the following features: assists, steals, blocks, turnovers, personal fouls

In [32]:
# create feature matrix (X) (it is a dataframe) that contains these columns from the nba dataframe:
# 'ast', 'stl', 'blk', 'tov', 'pf'
#feature_cols = ['ast', 'stl', 'blk', 'tov', 'pf']
#X = nba[feature_cols]
X = result.loc[:,'AccountCodeZZZ':'x4723']
print (X)

      AccountCodeZZZ  InOutCodeZZZ  ActBudCodeZZZ  payment  thank  deposit  \
0                 10             1              1      1.0    1.0      0.0   
1                 21             2              1      0.0    0.0      1.0   
2                 20             3              1      0.0    0.0      1.0   
3                 20             4              1      0.0    0.0      1.0   
4                 21             3              1      0.0    0.0      1.0   
5                 21             3              1      0.0    0.0      1.0   
6                  5             3              1      0.0    0.0      0.0   
7                  2             3              1      1.0    1.0      0.0   
8                  7             3              1      0.0    0.0      0.0   
9                  5             4              1      0.0    0.0      0.0   
10                 4             1              1      1.0    0.0      0.0   
11                 7             3              1      0.0    0.

In [ ]:
#X1 = X.isnull().values.any()
#X1

In [33]:
# create response vector (y) that contains the position numbers (our labels)
#y=nba.pos_num
y=result.CatBillCodeZZZ
print y

0         1
1         2
2         3
3         4
4         3
5         3
6         3
7         5
8         3
9         6
10        7
11        3
12        8
13        9
14        3
15       10
16       11
17       10
18       11
19       10
20       11
21       10
22       12
23       10
24       11
25       10
26       11
27       10
28       11
29       10
       ... 
4602    262
4603     39
4604     65
4605     84
4606    151
4607    270
4608    270
4609     39
4610    278
4611     53
4612     84
4613     84
4614     39
4615    130
4616     11
4617     84
4618    272
4619    280
4620     53
4621     53
4622     10
4623    273
4624    121
4625    270
4626    270
4627     57
4628     53
4629     53
4630     53
4631     53
Name: CatBillCodeZZZ, dtype: int64


In [34]:
X.shape


(4632, 5034)

In [35]:
y.shape

(4632L,)

## Step 3: Train a KNN model (K=5)

In [36]:
# import KNN class we need from scikit-learn
from sklearn.neighbors import KNeighborsClassifier

In [37]:
# instantiate the estimator with K=5
knn = KNeighborsClassifier(n_neighbors=5)
type(knn)

sklearn.neighbors.classification.KNeighborsClassifier

In [38]:
# run a knn.fit on the data to build the model
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

## Step 4: Use the model:
## Predict observation category

Predict for the category for the transactions on the following months

In [39]:
path = 'C:/Users/saang/Documents/Home_Metrics/'
url = path + 'Transactions_2016_07_and_08.csv'
ntran = pd.read_csv(url)

In [40]:
ndesc= ntran['Description']
ndescdf = pd.DataFrame(ndesc)
ndescdf.Description = ndescdf.Description.str.lower()

import unicodedata

def remove_accents(input_str):
    decoded_str = input_str.decode("windows-1252")
    nfkd_form = unicodedata.normalize('NFKD', unicode(decoded_str))
#    nfkd_form = unicodedata.normalize('NFKD', unicode(input_str))
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii


from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\s+', gaps=True)
#tokens_lower = [map(string.lower, input_string)]

for index, row in ndescdf.iterrows():
    ndescdf.loc[index,'Description'] = remove_accents(ndescdf.loc[index,'Description'])
    ndescdf.loc[index,'Description'] = tokenizer.tokenize(ndescdf.loc[index,'Description'])

#ndescdf

In [41]:
import string
# list found at http://www.textfixer.com/resources/common-english-words.txt
# 'not' has been removed (do you know why ?)

stopwords_ = "a,able,about,across,after,all,almost,also,am,among,an,and,any,\
are,as,at,be,because,been,but,by,can,could,dear,did,do,does,either,\
else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,\
how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,\
me,might,most,must,my,neither,no,of,off,often,on,only,or,other,our,\
own,rather,said,say,says,she,should,since,so,some,than,that,the,their,\
them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,\
what,when,where,which,while,who,whom,why,will,with,would,yet,you,your,%%]".split(',')

#print("--- stopwords in english: {}".format(stopwords_))

#punctuation_ = set(string.punctuation)
#print("--- punctuation: {}".format(string.punctuation))

def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

for index, row in ndescdf.iterrows():
    ndescdf.loc[index,'Description'] = filter_tokens(ndescdf.loc[index,'Description'])
    #return([w for w in descdf.loc[index,'Description'] if not w in stopwords_ and not w in punctuation_])
    
#ndescdf

# Create a zero values Data Frame

In [42]:
import numpy as np
#df.shape
a = np.zeros((441,5031))

ndf=pd.DataFrame(data=a,columns=features.loc[0,])
ndf.shape
#df.head()

(441, 5031)

In [43]:
#frames=[ntran, ndf]
#df1 = pd.concat(frames, )
nresult = pd.concat([ntran, ndf], axis=1)
#result

In [44]:

#wordlist = wordstring.split()
#var2=['payment', 'thank']
#wordlist=var2

wordfreq = []
wordlist = ""
def rocks(wordlist):
    for w in wordlist:
        wordfreq.append(wordlist.count(w))
    return (str(zip(wordlist, wordfreq)))
    
countspw = []
for desc in ndescdf.Description:
    countspw.append(rocks(desc))
#   print desc


# check that the DataFrame and the list of years are the same length
assert (len(ndescdf) == len(countspw))

#print countspw

# save that list as a new column
nresult['countspw'] = countspw

nresult.head()
nresult.shape

(441, 5048)

In [45]:
nresult.head()

,Date,Category,Description,Bill,Account,In/Out,Actuals/Budget,addt'l info,CatBill,CatBillCode2,...,9531,x6506,x6128,06/30/16,0737,06/29/16,caff,umbria,x4723,countspw
0,7/1/2016,Restaurants/Dining,CHECKCARD 0629 BIG FISH GRILL ...,NaN,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,BIG FISH GRILL,Restaurants/Dining:N/A:PAYMENT,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('checkcard', 1), ('0629', 1), ('big', 1), ('..."
1,7/1/2016,Home Maintenance,Greystone HOA Bill Payment,HOA,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,NaN,Home Maintenance:HOA:PAYMENT,261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('greystone', 1), ('hoa', 1), ('bill', 1), ('..."
2,7/1/2016,Home Improvement,HOME DEPOT CREDIT SERVICES Bill Payment,NaN,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,Home Depot,Home Improvement:N/A:PAYMENT,107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('home', 1), ('depot', 1), ('credit', 1), ('s..."
3,7/1/2016,Investment Income,Online Banking Transfer Conf# t9ok6iw73; ...,Cirillo,Bank of America - Bank - Bank of America Core ...,INCOME,Actuals,Transfer,Investment Income:Cirillo:INCOME,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('online', 1), ('banking', 1), ('transfer', 1..."
4,7/1/2016,PAYPAL,PAYPAL *LATINAMERIC xxxxxxxxxx xxxxxxxx xxx...,NaN,American Express Cards - Business Gold Rewards...,PAYMENT,Actuals,PayPal,PAYPAL:N/A:PAYMENT,274,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('paypal', 1), ('*latinameric', 1), ('xxxxxxx..."


In [46]:
import numpy as np
from itertools import chain
from numpy import ndenumerate 
import ast

#result5 = result.head(5)

line=list()
tup=()
for index, row in nresult.iterrows():
    #line = np.array(row.countspw)
    #line = row.countspw
    items = list(ast.literal_eval(row.countspw))

    for item in items:
        #print(item)
        tup = tuple(item)
      
        for i,j in enumerate(item):
            #print index ,i, j
            col = str(item[0])
            #print ("colum is " +str(item[0]))
            val = item[1]
            #print ("value is "+ str(item[1]))
            if col in nresult.columns:
                nresult.loc[index,col] = val
            else:
                print ("Index =: " +str(index) + " Colum is =: " + col + " Value is =: "+ str(val))
                #print ("colum is " + col)
                #print ("value is "+ str(val))
            if (index == 1000):
                print "1000"
            if (index == 2000):
                print "2000"
            if (index == 3000):
                print "3000"
            if (index == 4000):
                print "4000"

Index =: 0 Colum is =: fish Value is =: 1
Index =: 0 Colum is =: fish Value is =: 1
Index =: 0 Colum is =: woodinville Value is =: 1
Index =: 0 Colum is =: woodinville Value is =: 1
Index =: 0 Colum is =: xxxxxxxxxxxxxxxxxxx5983 Value is =: 1
Index =: 0 Colum is =: xxxxxxxxxxxxxxxxxxx5983 Value is =: 1
Index =: 1 Colum is =: greystone Value is =: 1
Index =: 1 Colum is =: greystone Value is =: 1
Index =: 2 Colum is =: services Value is =: 1
Index =: 2 Colum is =: services Value is =: 1
Index =: 3 Colum is =: online Value is =: 1
Index =: 3 Colum is =: online Value is =: 1
Index =: 3 Colum is =: banking Value is =: 1
Index =: 3 Colum is =: banking Value is =: 1
Index =: 3 Colum is =: t9ok6iw73; Value is =: 1
Index =: 3 Colum is =: t9ok6iw73; Value is =: 1
Index =: 3 Colum is =: glover, Value is =: 1
Index =: 3 Colum is =: glover, Value is =: 1
Index =: 3 Colum is =: russell Value is =: 1
Index =: 3 Colum is =: russell Value is =: 1
Index =: 4 Colum is =: *latinameric Value is =: 1
Index 

In [47]:
nresult.loc[:,'payment'].head(50)


0     0.0
1     1.0
2     1.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    1.0
18    1.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    1.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    1.0
35    0.0
36    1.0
37    1.0
38    0.0
39    0.0
40    1.0
41    0.0
42    0.0
43    0.0
44    0.0
45    1.0
46    1.0
47    0.0
48    0.0
49    1.0
Name: payment, dtype: float64

In [48]:
l=list()
for index, row in nresult.iterrows():
    l = result.loc[index,'AccountCodeZZZ':'x4723']
    l = list(l)
    nresult.loc[index,'newcatZZZ'] = knn.predict(l)
    #print l
    
#X = result.loc[:,'dayZZZ':'xxxx4723']

C:\Program Files\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Program Files\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Program Files\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

In [49]:
nresult.head()

,Date,Category,Description,Bill,Account,In/Out,Actuals/Budget,addt'l info,CatBill,CatBillCode2,...,x6506,x6128,06/30/16,0737,06/29/16,caff,umbria,x4723,countspw,newcatZZZ
0,7/1/2016,Restaurants/Dining,CHECKCARD 0629 BIG FISH GRILL ...,NaN,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,BIG FISH GRILL,Restaurants/Dining:N/A:PAYMENT,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('checkcard', 1), ('0629', 1), ('big', 1), ('...",1.0
1,7/1/2016,Home Maintenance,Greystone HOA Bill Payment,HOA,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,NaN,Home Maintenance:HOA:PAYMENT,261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('greystone', 1), ('hoa', 1), ('bill', 1), ('...",3.0
2,7/1/2016,Home Improvement,HOME DEPOT CREDIT SERVICES Bill Payment,NaN,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,Home Depot,Home Improvement:N/A:PAYMENT,107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('home', 1), ('depot', 1), ('credit', 1), ('s...",3.0
3,7/1/2016,Investment Income,Online Banking Transfer Conf# t9ok6iw73; ...,Cirillo,Bank of America - Bank - Bank of America Core ...,INCOME,Actuals,Transfer,Investment Income:Cirillo:INCOME,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('online', 1), ('banking', 1), ('transfer', 1...",4.0
4,7/1/2016,PAYPAL,PAYPAL *LATINAMERIC xxxxxxxxxx xxxxxxxx xxx...,NaN,American Express Cards - Business Gold Rewards...,PAYMENT,Actuals,PayPal,PAYPAL:N/A:PAYMENT,274,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[('paypal', 1), ('*latinameric', 1), ('xxxxxxx...",3.0


In [51]:
cols=['Date','Category','Description','Bill','Account','In/Out','Actuals/Budget','CatBill','CatBillCode2','newcatZZZ']
nfinal=nresult[cols]

In [52]:
nfinal

,Date,Category,Description,Bill,Account,In/Out,Actuals/Budget,CatBill,CatBillCode2,newcatZZZ
0,7/1/2016,Restaurants/Dining,CHECKCARD 0629 BIG FISH GRILL ...,NaN,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,Restaurants/Dining:N/A:PAYMENT,53,1.0
1,7/1/2016,Home Maintenance,Greystone HOA Bill Payment,HOA,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,Home Maintenance:HOA:PAYMENT,261,3.0
2,7/1/2016,Home Improvement,HOME DEPOT CREDIT SERVICES Bill Payment,NaN,Bank of America - Bank - Bank of America Core ...,PAYMENT,Actuals,Home Improvement:N/A:PAYMENT,107,3.0
3,7/1/2016,Investment Income,Online Banking Transfer Conf# t9ok6iw73; ...,Cirillo,Bank of America - Bank - Bank of America Core ...,INCOME,Actuals,Investment Income:Cirillo:INCOME,27,4.0
4,7/1/2016,PAYPAL,PAYPAL *LATINAMERIC xxxxxxxxxx xxxxxxxx xxx...,NaN,American Express Cards - Business Gold Rewards...,PAYMENT,Actuals,PAYPAL:N/A:PAYMENT,274,3.0
5,7/1/2016,Interest,PURCHASE INTEREST CHARGE,ChaseCC,Chase - Credit Card - S. ANGULO,PAYMENT,Actuals,Interest:ChaseCC:PAYMENT,166,3.0
6,7/1/2016,Groceries,Withdrawal #xxxxxxxxxx81 - COSTCO WHSE #0747 W...,NaN,State Department FCU - Bank - BASIC CHECKING,PAYMENT,Actuals,Groceries:N/A:PAYMENT,84,3.0
7,7/1/2016,Groceries,Withdrawal #xxxxxxxxxx92 - COSTCO WHSE #0747 W...,NaN,State Department FCU - Bank - BASIC CHECKING,PAYMENT,Actuals,Groceries:N/A:PAYMENT,84,11.0
8,7/2/2016,Interest,INTEREST CHARGE ON BALANCE TRANSFERS,Discover,Discover Card - Discover,PAYMENT,Actuals,Interest:Discover:PAYMENT,172,3.0
9,7/2/2016,Interest,INTEREST CHARGE ON BALANCE TRANSFERS,Discover,Discover Card - Discover,PAYMENT,Actuals,Interest:Discover:PAYMENT,172,6.0


In [53]:
path = 'C:/Users/saang/Documents/Home_Metrics/'
url = path + 'Transactions_2016_07_and_08_prediction.csv'
nfinal.to_csv(url, sep='\t', encoding='utf-8')

In [ ]:
# create a list to represent a player
player = [[1, 1, 0, 1, 2]]

In [ ]:
# make a prediction with the model
knn.predict(player)

In [ ]:
# calculate the predicted probabilities
knn.predict_proba(player)

## Step 5: Repeat steps 3 and 4 using K=50

In [ ]:
# repeat for K=50
knn = KNeighborsClassifier(n_neighbors=50)
type(knn)
knn.fit(X, y)

In [ ]:
# calculate predicted probabilities
knn.predict_proba(player)

## Bonus: Explore the features to ones ones that likely are the most predictive

In [ ]:
# allow plots to appear in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 14

In [ ]:
# get a count of assists grouped by position.  Use .unstack() to format the output

#nba.groupby('pos').ast.describe()
nba.groupby('pos').ast.describe().unstack()

In [ ]:
# allow plots to appear in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 14

# create a custom colormap
from matplotlib.colors import ListedColormap
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

In [ ]:
# box plot of assists grouped by position
nba.boxplot(column='ast', by='pos') 

In [ ]:
# group plot of histograms of assists grouped by position
nba.plot()